# Key-Value Store Performance Testing

このノートブックは各Key-Valueストアサービスの性能テストを自動化します。

## 対象サービス
1. **1-coordinator-ring**: コーディネーターリング実装
2. **2-quorum-consistency**: クォーラム一貫性実装
3. **3-sharding-replica**: シャーディング・レプリカ実装
4. **4-distributed-lock**: 分散ロック実装
5. **5-cache-aside**: キャッシュアサイド実装
6. **6-bloom-sstable**: BloomフィルタとSSTable実装
7. **7-rate-limiting**: レートリミティング実装
8. **8-line-streams**: Line風ストリーム実装
9. **9-session-store**: セッションストア実装
10. **10-leaderboard**: リーダーボード実装

## テストパターン
- **軽負荷**: 10 clients, 1000 requests
- **中負荷**: 50 clients, 10000 requests  
- **高負荷**: 100 clients, 50000 requests

In [1]:
import time
import json
import subprocess
import requests
import statistics
import pandas as pd
import numpy as np
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
from pathlib import Path
import docker
from contextlib import contextmanager

# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# パス設定
BASE_DIR = Path("/Users/codefox/workspace/practice_infra_arch/key_value_store")
DATA_DIR = BASE_DIR / "analysis" / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("✅ 必要なライブラリがインポートされました")

✅ 必要なライブラリがインポートされました


In [2]:
# サービス設定
SERVICES = {
    "1-coordinator-ring": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "2-quorum-consistency": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "3-sharding-replica": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "4-distributed-lock": {
        "port": 8000,
        "endpoints": {
            "health": "/stats",
            "acquire": "/acquire",
            "release": "/release"
        },
        "test_data": {"resource": "test_resource", "owner": "test_client"}
    },
    "5-cache-aside": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "get": "/get",
            "set": "/set"
        },
        "test_data": {"entity_type": "user", "entity_id": "1", "data": {"name": "test"}}
    },
    "6-bloom-sstable": {
        "port": 5000,
        "endpoints": {
            "health": "/health",
            "write": "/write",
            "read": "/read"
        },
        "test_data": {"key": "test_key", "value": "test_value"}
    },
    "7-rate-limiting": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "check": "/check_rate"
        },
        "test_data": {"user_id": "test_user"}
    },
    "8-line-streams": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "produce": "/produce",
            "consume": "/consume"
        },
        "test_data": {"message": "test_message", "consumer": "test_consumer"}
    },
    "9-session-store": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "login": "/login",
            "me": "/me"
        },
        "test_data": {"username": "test_user"}
    },
    "10-leaderboard": {
        "port": 8000,
        "endpoints": {
            "health": "/health",
            "score": "/score",
            "top": "/top/10"
        },
        "test_data": {"user_id": "test_user", "score": 100}
    }
}

In [3]:
TEST_PATTERNS = {
    "light": {
        "description": "軽負荷",
        "clients": 10,
        "requests": 1000,
        "timeout": 30,
        "ramp_up": 5
    },
    "medium": {
        "description": "中負荷",
        "clients": 50, 
        "requests": 10000,
        "timeout": 60,
        "ramp_up": 10
    },
    "heavy": {
        "description": "高負荷",
        "clients": 100,
        "requests": 50000,
        "timeout": 120,
        "ramp_up": 20
    }
}

In [12]:
class DockerManager:
    """Dockerコンテナの管理クラス"""
    
    def __init__(self, service_name):
        self.service_name = service_name
        self.service_dir = BASE_DIR / service_name
        self.client = docker.from_env()
        
    def start_service(self):
        """サービスを起動"""
        logger.info(f"🚀 {self.service_name} サービスを起動中...")
        try:
            cmd = ["docker", "compose", "up", "-d", "--build"]
            result = subprocess.run(
                cmd, 
                cwd=self.service_dir, 
                capture_output=True, 
                text=True,
                timeout=300  # 5分でタイムアウト
            )
            
            if result.returncode != 0:
                logger.error(f"❌ {self.service_name} 起動失敗: {result.stderr}")
                return False
                
            logger.info(f"✅ {self.service_name} サービス起動完了")
            return True
            
        except subprocess.TimeoutExpired:
            logger.error(f"❌ {self.service_name} 起動タイムアウト")
            return False
        except Exception as e:
            logger.error(f"❌ {self.service_name} 起動中にエラー: {str(e)}")
            return False
    
    def stop_service(self):
        """サービスを停止"""
        logger.info(f"🛑 {self.service_name} サービスを停止中...")
        try:
            cmd = ["docker", "compose", "down", "-v"]
            result = subprocess.run(
                cmd, 
                cwd=self.service_dir, 
                capture_output=True, 
                text=True,
                timeout=60
            )
            
            if result.returncode == 0:
                logger.info(f"✅ {self.service_name} サービス停止完了")
            else:
                logger.warning(f"⚠️ {self.service_name} 停止時に警告: {result.stderr}")
                
        except Exception as e:
            logger.error(f"❌ {self.service_name} 停止中にエラー: {str(e)}")
    
    def health_check(self, max_retries=30):
        """ヘルスチェック"""
        service_config = SERVICES[self.service_name]
        health_url = f"http://localhost:{service_config['port']}{service_config['endpoints']['health']}"
        
        for attempt in range(max_retries):
            try:
                response = requests.get(health_url, timeout=5)
                if response.status_code == 200:
                    logger.info(f"✅ {self.service_name} ヘルスチェック成功")
                    return True
            except requests.exceptions.RequestException:
                pass
                
            time.sleep(2)
            logger.info(f"⏳ {self.service_name} ヘルスチェック待機中... ({attempt + 1}/{max_retries})")
        
        logger.error(f"❌ {self.service_name} ヘルスチェック失敗")
        return False
    
    def get_container_stats(self):
        """コンテナの統計情報を取得"""
        try:
            containers = self.client.containers.list(filters={"label": f"com.docker.compose.project={self.service_name.replace('-', '')}"})
            stats = {}
            
            for container in containers:
                container_stats = container.stats(stream=False)
                stats[container.name] = {
                    "cpu_percent": self._calculate_cpu_percent(container_stats),
                    "memory_usage": container_stats["memory"]["usage"],
                    "memory_limit": container_stats["memory"]["limit"]
                }
            return stats
        except Exception as e:
            logger.warning(f"⚠️ {self.service_name} 統計情報取得失敗: {str(e)}")
            return {}
    
    def _calculate_cpu_percent(self, stats):
        """CPU使用率を計算"""
        try:
            cpu_delta = stats["cpu_stats"]["cpu_usage"]["total_usage"] - stats["precpu_stats"]["cpu_usage"]["total_usage"]
            system_delta = stats["cpu_stats"]["system_cpu_usage"] - stats["precpu_stats"]["system_cpu_usage"]
            num_cpus = len(stats["cpu_stats"]["cpu_usage"]["percpu_usage"])
            
            if system_delta > 0:
                return (cpu_delta / system_delta) * num_cpus * 100.0
        except (KeyError, ZeroDivisionError):
            pass
        return 0.0

@contextmanager
def service_context(service_name):
    """サービスのライフサイクル管理コンテキストマネージャー"""
    docker_manager = DockerManager(service_name)
    docker_manager.stop_service()
    
    try:
        # サービス起動
        if not docker_manager.start_service():
            raise RuntimeError(f"Failed to start {service_name}")
        
        # ヘルスチェック
        if not docker_manager.health_check():
            raise RuntimeError(f"Health check failed for {service_name}")
        
        yield docker_manager
        
    finally:
        # サービス停止
        docker_manager.stop_service()
        # クールタイム: コンテナの完全解放 / ポート解放待ち
        logger.info(f"⏳ {service_name} 停止後のクールタイム: 60秒待機します...")
        time.sleep(60)

print("✅ Docker管理クラスが定義されました")

✅ Docker管理クラスが定義されました


In [5]:
class PerformanceTester:
    """性能テスト実行クラス"""
    
    def __init__(self, service_name):
        self.service_name = service_name
        self.service_config = SERVICES[service_name]
        self.base_url = f"http://localhost:{self.service_config['port']}"
        self.results = []
        
    def run_single_request(self, endpoint, method="GET", data=None):
        """単一リクエストの実行と測定"""
        url = f"{self.base_url}{endpoint}"
        start_time = time.time()
        
        try:
            if method == "POST":
                response = requests.post(url, json=data, timeout=10)
            else:
                response = requests.get(url, params=data, timeout=10)
            
            end_time = time.time()
            response_time = (end_time - start_time) * 1000  # ミリ秒
            
            return {
                "response_time": response_time,
                "status_code": response.status_code,
                "success": 200 <= response.status_code < 300,
                "error": None
            }
            
        except Exception as e:
            end_time = time.time()
            response_time = (end_time - start_time) * 1000
            
            return {
                "response_time": response_time,
                "status_code": 0,
                "success": False,
                "error": str(e)
            }
    
    def run_load_test(self, pattern_name, docker_manager):
        """負荷テストの実行"""
        pattern = TEST_PATTERNS[pattern_name]
        clients = pattern["clients"]
        total_requests = pattern["requests"]
        requests_per_client = total_requests // clients
        
        logger.info(f"🧪 {self.service_name} - {pattern['description']} 開始")
        logger.info(f"   クライアント数: {clients}, 総リクエスト数: {total_requests}")
        
        # テスト対象エンドポイントの決定
        test_endpoint, test_method, test_data = self._get_test_endpoint()
        
        # 初期統計情報取得
        initial_stats = docker_manager.get_container_stats()
        
        start_time = time.time()
        all_results = []
        
        # 並行テスト実行
        with ThreadPoolExecutor(max_workers=clients) as executor:
            futures = []
            
            for client_id in range(clients):
                future = executor.submit(
                    self._client_worker, 
                    client_id, 
                    requests_per_client, 
                    test_endpoint, 
                    test_method, 
                    test_data
                )
                futures.append(future)
            
            # 結果収集
            for future in as_completed(futures):
                try:
                    client_results = future.result()
                    all_results.extend(client_results)
                except Exception as e:
                    logger.error(f"❌ クライアントワーカーエラー: {str(e)}")
        
        end_time = time.time()
        total_duration = end_time - start_time
        
        # 最終統計情報取得
        final_stats = docker_manager.get_container_stats()
        
        # 結果分析
        analysis = self._analyze_results(all_results, total_duration, initial_stats, final_stats)
        analysis.update({
            "service_name": self.service_name,
            "pattern_name": pattern_name,
            "pattern_description": pattern["description"],
            "timestamp": datetime.now().isoformat(),
            "clients": clients,
            "total_requests": len(all_results),
            "duration": total_duration
        })
        
        logger.info(f"✅ {self.service_name} - {pattern['description']} 完了")
        logger.info(f"   実行時間: {total_duration:.2f}秒, QPS: {analysis['qps']:.2f}")
        
        return analysis
    
    def _get_test_endpoint(self):
        """サービスごとの適切なテストエンドポイントを決定"""
        endpoints = self.service_config["endpoints"]
        test_data = self.service_config["test_data"]
        
        if "write" in endpoints:
            return endpoints["write"], "POST", test_data
        elif "put" in endpoints:
            return endpoints["put"], "POST", test_data
        elif "publish" in endpoints:
            return endpoints["publish"], "POST", test_data
        elif "produce" in endpoints:
            return endpoints["produce"], "POST", test_data
        elif "acquire" in endpoints:
            return endpoints["acquire"], "POST", test_data
        else:
            return endpoints["health"], "GET", None
    
    def _client_worker(self, client_id, requests_count, endpoint, method, data):
        """クライアントワーカー（各クライアントが実行する処理）"""
        results = []
        
        for i in range(requests_count):
            # データの動的生成（一意性確保）
            if data:
                test_data = data.copy()
                if "key" in test_data:
                    test_data["key"] = f"{test_data['key']}_{client_id}_{i}"
                if "resource" in test_data:
                    test_data["resource"] = f"{test_data['resource']}_{client_id}_{i}"
                if "client_id" in test_data:
                    test_data["client_id"] = f"{test_data['client_id']}_{client_id}"
            else:
                test_data = None
            
            result = self.run_single_request(endpoint, method, test_data)
            result["client_id"] = client_id
            result["request_id"] = i
            results.append(result)
        
        return results
    
    def _analyze_results(self, results, duration, initial_stats, final_stats):
        """結果分析"""
        successful_results = [r for r in results if r["success"]]
        response_times = [r["response_time"] for r in successful_results]
        
        if not response_times:
            return {
                "total_requests": len(results),
                "successful_requests": 0,
                "error_rate": 100.0,
                "qps": 0.0,
                "avg_response_time": 0.0,
                "median_response_time": 0.0,
                "p95_response_time": 0.0,
                "p99_response_time": 0.0,
                "min_response_time": 0.0,
                "max_response_time": 0.0
            }
        
        return {
            "total_requests": len(results),
            "successful_requests": len(successful_results),
            "error_rate": ((len(results) - len(successful_results)) / len(results)) * 100,
            "qps": len(successful_results) / duration,
            "avg_response_time": statistics.mean(response_times),
            "median_response_time": statistics.median(response_times),
            "p95_response_time": np.percentile(response_times, 95),
            "p99_response_time": np.percentile(response_times, 99),
            "min_response_time": min(response_times),
            "max_response_time": max(response_times),
            "initial_stats": initial_stats,
            "final_stats": final_stats
        }

print("✅ 性能テストクラスが定義されました")

✅ 性能テストクラスが定義されました


In [ ]:
def run_comprehensive_test(selected_services=None, selected_patterns=None):
    """包括的な性能テストの実行"""
    if selected_services is None:
        selected_services = list(SERVICES.keys())
    if selected_patterns is None:
        selected_patterns = list(TEST_PATTERNS.keys())
    
    all_results = []
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    logger.info(f"🚀 包括的性能テスト開始 - {timestamp}")
    logger.info(f"対象サービス: {selected_services}")
    logger.info(f"テストパターン: {selected_patterns}")
    
    total_tests = len(selected_services) * len(selected_patterns)
    current_test = 0
    
    for service_name in selected_services:
        logger.info(f"\n📊 {service_name} のテスト開始")
        
        try:
            with service_context(service_name) as docker_manager:
                tester = PerformanceTester(service_name)
                
                for pattern_name in selected_patterns:
                    current_test += 1
                    logger.info(f"\n進行状況: {current_test}/{total_tests}")
                    
                    try:
                        result = tester.run_load_test(pattern_name, docker_manager)
                        all_results.append(result)
                        
                        # 中間結果保存
                        save_results([result], f"intermediate_{service_name}_{pattern_name}_{timestamp}")
                        
                        # テスト間の休憩
                        if current_test < total_tests:
                            logger.info("⏸️ 5秒間の休憩...")
                            time.sleep(5)
                            
                    except Exception as e:
                        logger.error(f"❌ {service_name} - {pattern_name} テスト失敗: {str(e)}")
                        # エラー結果も記録
                        error_result = {
                            "service_name": service_name,
                            "pattern_name": pattern_name,
                            "error": str(e),
                            "timestamp": datetime.now().isoformat()
                        }
                        all_results.append(error_result)
                        
        except Exception as e:
            logger.error(f"❌ {service_name} サービス全体のテスト失敗: {str(e)}")
            continue
    
    # 最終結果保存
    save_results(all_results, f"comprehensive_test_{timestamp}")
    
    logger.info(f"\n🎉 全テスト完了! 結果は {DATA_DIR} に保存されました")
    return all_results

def save_results(results, filename_prefix):
    """結果をファイルに保存"""
    # JSON形式で保存
    json_file = DATA_DIR / f"{filename_prefix}.json"
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    # CSV形式で保存（数値データのみ）
    csv_file = DATA_DIR / f"{filename_prefix}.csv"
    
    if results and not any('error' in r for r in results):
        # エラーがない場合のみCSV出力
        df = pd.DataFrame(results)
        # 複雑なオブジェクトを除外
        numeric_columns = df.select_dtypes(include=[np.number, 'object']).columns
        simple_columns = [col for col in numeric_columns 
                         if col not in ['initial_stats', 'final_stats']]
        df[simple_columns].to_csv(csv_file, index=False)
    
    logger.info(f"💾 結果保存完了: {json_file}")

print("✅ テスト実行関数が定義されました")

✅ テスト実行関数が定義されました


In [ ]:
# ユーザーインターフェース用関数

def show_services():
    """利用可能なサービス一覧を表示"""
    print("📋 利用可能なサービス:")
    for i, (name, config) in enumerate(SERVICES.items(), 1):
        print(f"  {i}. {name} (ポート: {config['port']})")

def show_patterns():
    """利用可能なテストパターン一覧を表示"""
    print("📋 利用可能なテストパターン:")
    for i, (name, pattern) in enumerate(TEST_PATTERNS.items(), 1):
        print(f"  {i}. {name}: {pattern['description']} ({pattern['clients']} clients, {pattern['requests']} requests)")

def quick_test(service_name, pattern_name="light"):
    """単一サービスの簡易テスト"""
    if service_name not in SERVICES:
        print(f"❌ サービス '{service_name}' が見つかりません")
        show_services()
        return None
    
    if pattern_name not in TEST_PATTERNS:
        print(f"❌ テストパターン '{pattern_name}' が見つかりません")
        show_patterns()
        return None
    
    return run_comprehensive_test([service_name], [pattern_name])

def test_all_services(pattern_name="light"):
    """全サービスの指定パターンでのテスト"""
    if pattern_name not in TEST_PATTERNS:
        print(f"❌ テストパターン '{pattern_name}' が見つかりません")
        show_patterns()
        return None
    
    return run_comprehensive_test(None, [pattern_name])

def test_service_all_patterns(service_name):
    """指定サービスの全パターンでのテスト"""
    if service_name not in SERVICES:
        print(f"❌ サービス '{service_name}' が見つかりません")
        show_services()
        return None
    
    return run_comprehensive_test([service_name], None)

print("✅ ユーザーインターフェース関数が定義されました")
print("\n🎯 使用方法:")
print("  - show_services(): サービス一覧表示")
print("  - show_patterns(): テストパターン一覧表示")
print("  - quick_test('service_name', 'pattern_name'): 単一サービステスト")
print("  - test_all_services('pattern_name'): 全サービステスト")
print("  - test_service_all_patterns('service_name'): 指定サービス全パターンテスト")
print("  - run_comprehensive_test(): 完全なカスタムテスト")

✅ ユーザーインターフェース関数が定義されました

🎯 使用方法:
  - show_services(): サービス一覧表示
  - show_patterns(): テストパターン一覧表示
  - quick_test('service_name', 'pattern_name'): 単一サービステスト
  - test_all_services('pattern_name'): 全サービステスト
  - test_service_all_patterns('service_name'): 指定サービス全パターンテスト
  - run_comprehensive_test(): 完全なカスタムテスト


## 🚀 テスト実行例

以下のセルを実行してテストを開始してください。テスト前に必要に応じて設定を確認・変更できます。

In [8]:
# 利用可能なオプションを表示
show_services()
print()
show_patterns()

📋 利用可能なサービス:
  1. 1-coordinator-ring (ポート: 8000)
  2. 2-quorum-consistency (ポート: 8000)
  3. 3-sharding-replica (ポート: 8000)
  4. 4-distributed-lock (ポート: 8000)
  5. 5-cache-aside (ポート: 8000)
  6. 6-bloom-sstable (ポート: 5000)
  7. 7-rate-limiting (ポート: 8000)
  8. 8-line-streams (ポート: 8000)
  9. 9-session-store (ポート: 8000)
  10. 10-leaderboard (ポート: 8000)

📋 利用可能なテストパターン:
  1. light: 軽負荷 (10 clients, 1000 requests)
  2. medium: 中負荷 (50 clients, 10000 requests)
  3. heavy: 高負荷 (100 clients, 50000 requests)


In [13]:
#   1. 1-coordinator-ring
#   2. 2-quorum-consistency
#   3. 3-sharding-replica
#   4. 4-distributed-lock
#   5. 5-cache-aside
#   6. 6-bloom-sstable
#   7. 7-rate-limiting
#   8. 8-line-streams
#   9. 9-session-store
#   10. 10-leaderboard

# 例: 単一サービスの軽負荷テスト
quick_test("2-quorum-consistency", "light")

2025-09-08 21:41:55,142 - INFO - 🚀 包括的性能テスト開始 - 20250908_214155
2025-09-08 21:41:55,143 - INFO - 対象サービス: ['2-quorum-consistency']
2025-09-08 21:41:55,143 - INFO - テストパターン: ['light']
2025-09-08 21:41:55,144 - INFO - 
📊 2-quorum-consistency のテスト開始
2025-09-08 21:41:55,179 - INFO - 🛑 2-quorum-consistency サービスを停止中...
2025-09-08 21:41:55,261 - INFO - ✅ 2-quorum-consistency サービス停止完了
2025-09-08 21:41:55,262 - INFO - 🚀 2-quorum-consistency サービスを起動中...
2025-09-08 21:41:57,148 - ERROR - ❌ 2-quorum-consistency 起動失敗: time="2025-09-08T21:41:55+09:00" level=warning msg="/Users/codefox/workspace/practice_infra_arch/key_value_store/2-quorum-consistency/compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 2-quorum-consistency-app  Built
 Network 2-quorum-consistency_default  Creating
 Network 2-quorum-consistency_default  Created
 Container 2-quorum-consistency-redis-node1-1  Creating
 Container 2-quorum-consistency-redis-node2-1  Creatin

[]

In [10]:
# 例: 全サービスの軽負荷テスト
test_all_services("light")

2025-09-08 21:35:34,738 - INFO - 🚀 包括的性能テスト開始 - 20250908_213534
2025-09-08 21:35:34,738 - INFO - 対象サービス: ['1-coordinator-ring', '2-quorum-consistency', '3-sharding-replica', '4-distributed-lock', '5-cache-aside', '6-bloom-sstable', '7-rate-limiting', '8-line-streams', '9-session-store', '10-leaderboard']
2025-09-08 21:35:34,738 - INFO - テストパターン: ['light']
2025-09-08 21:35:34,739 - INFO - 
📊 1-coordinator-ring のテスト開始
2025-09-08 21:35:34,771 - INFO - 🚀 1-coordinator-ring サービスを起動中...
2025-09-08 21:35:43,427 - ERROR - ❌ 1-coordinator-ring 起動失敗: time="2025-09-08T21:35:34+09:00" level=warning msg="/Users/codefox/workspace/practice_infra_arch/key_value_store/1-coordinator-ring/compose.yaml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 1-coordinator-ring-app  Built
 Network 1-coordinator-ring_default  Creating
 Network 1-coordinator-ring_default  Created
 Container redis-node2  Creating
 Container redis-node3  Creating
 Container redi

KeyboardInterrupt: 

In [ ]:
# 例: 包括的テスト（全サービス、全パターン）
# ⚠️ 注意: このテストは非常に時間がかかります（1-2時間程度）
# results = run_comprehensive_test()

## 📊 結果について

テスト結果は以下の場所に保存されます：

- **JSONファイル**: `analysis/data/*.json` - 完全な結果データ
- **CSVファイル**: `analysis/data/*.csv` - 数値データのみ（分析用）

結果の分析には `analyze_results.ipynb` ノートブックを使用してください。

### 測定項目
- **レスポンス時間**: 平均、中央値、95%tile、99%tile
- **スループット**: QPS（Query Per Second）
- **エラー率**: 失敗したリクエストの割合
- **リソース使用量**: CPU、メモリ使用量（可能な場合）

### 注意事項
- テスト実行中は他のアプリケーションの使用を控えてください
- 高負荷テストは時間がかかる場合があります
- Docker環境が正常に動作することを事前に確認してください